In [1]:
import numpy as np
import h5py
import math
import os
import pathlib
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    BatchNormalization,
    Activation,
    Layer,
    ReLU,
    LeakyReLU,
)
from tensorflow.keras import backend as K

2022-09-21 19:13:17.672308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 19:13:17.781902: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-21 19:13:18.193568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-21 19:13:18.193615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
#from func import load_model, save_model

## Load dataset

In [3]:
folder = '../preprocessing-code/'
filename = "BKG_dataset.h5"

In [4]:
# make sure input data has correct input shape - background training data
with h5py.File(folder+filename, "r") as file:
    X_train = np.array(file["X_train"])
    X_test = np.array(file["X_test"])
    X_val = np.array(file["X_val"])

## Define Dense NN architecture

In [5]:
input_shape = 60
latent_dimension = 3
num_nodes = [16, 8]

In [6]:
# encoder
inputArray = Input(shape=(input_shape))
x = Dense(num_nodes[0], use_bias=False)(inputArray)
x = Activation("relu")(x)
x = Dense(latent_dimension, use_bias=False)(x)
encoder = Activation("relu")(x)

# decoder
x = Dense(num_nodes[0], use_bias=False)(encoder)
x = Activation("relu")(x)
decoder = Dense(input_shape)(x)

# create autoencoder
autoencoder = Model(inputs=inputArray, outputs=decoder)
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60)]              0         
                                                                 
 dense (Dense)               (None, 16)                960       
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 48        
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
 dense_2 (Dense)             (None, 16)                48        
                                                                 
 activation_2 (Activation)   (None, 16)                0     

2022-09-21 19:13:19.700795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 19:13:19.722107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 19:13:19.722269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 19:13:19.722670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
autoencoder.compile(optimizer=keras.optimizers.Adam(), loss="mse")

## Train model

In [8]:
EPOCHS = 10
BATCH_SIZE = 1024

In [9]:
history = autoencoder.fit(
    X_train,
    X_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, X_val),
)

2022-09-21 19:13:20.256684: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.
2022-09-21 19:13:20.768221: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.
2022-09-21 19:13:21.225056: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.
2022-09-21 19:13:21.390146: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/10
 121/2500 [>.............................] - ETA: 3s - loss: 91.4054  

2022-09-21 19:13:22.345594: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2500/2500 [==============================] - 5s 2ms/step - loss: 12.0560 - val_loss: 5.2271
Epoch 2/10
2500/2500 [==============================] - 3s 1ms/step - loss: 4.6259 - val_loss: 4.2053
Epoch 3/10
2500/2500 [==============================] - 3s 1ms/step - loss: 4.0238 - val_loss: 3.8670
Epoch 4/10
2500/2500 [==============================] - 3s 1ms/step - loss: 3.6051 - val_loss: 3.3665
Epoch 5/10
2500/2500 [==============================] - 3s 1ms/step - loss: 3.2140 - val_loss: 3.0474
Epoch 6/10
2500/2500 [==============================] - 3s 1ms/step - loss: 2.9319 - val_loss: 2.8204
Epoch 7/10
2500/2500 [==============================] - 3s 1ms/step - loss: 2.7714 - val_loss: 2.7011
Epoch 8/10
2500/2500 [==============================] - 3s 1ms/step - loss: 2.6506 - val_loss: 2.6153
Epoch 9/10
2500/2500 [==============================] - 3s 1ms/step - loss: 2.4913 - val_loss: 2.4170
Epoch 10/10
2500/2500 [==============================] - 3s 1ms/step - loss: 2.3224 - val_lo

In [10]:
model_name = "model_name"
model_directory = ""
#save_model(model_directory + model_name, autoencoder)

## Prediction - background

In [11]:
bkg_prediction = autoencoder.predict(X_test)

25000/25000 [==============================] - 23s 906us/step


2022-09-21 19:14:38.705182: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 195.3KiB (rounded to 200192)requested by op ConcatV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-09-21 19:14:38.705452: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-09-21 19:14:38.705466: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 39, Chunks in use: 39. 9.8KiB allocated for chunks. 9.8KiB in use in bin. 2.0KiB client-requested in use in bin.
2022-09-21 19:14:38.705474: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-09-21 19:14:38.705480: I tensorflow/core/commo

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Prediction - signals

In [ ]:
# add correct signal labels
signal_labels = ["signal_1", "signal_2", "signal_3", "signal_4"]

In [ ]:
# add correct path to signal files
signals_file = ["Ato4l_lepFilter_13TeV_dataset.h5", "hChToTauNu_13TeV_PU20_dataset.h5", "hToTauTau_13TeV_PU20_dataset.h5", "leptoquark_LOWMASS_lepFilter_13TeV_dataset.h5"]

In [ ]:
# read signal data
signal_data = []
for i, label in enumerate(signal_labels):
    with h5py.File(folder+signals_file[i], "r") as file:
        test_data = np.array(file["Data"])
    signal_data.append(test_data)

In [ ]:
signal_results = []

for i, label in enumerate(signal_labels):
    signal_prediction = autoencoder.predict(signal_data[i])
    signal_results.append(
        [label, signal_data[i], signal_prediction]
    )  # save [label, true, prediction] for signal

## Save results

In [ ]:
save_file = "save_file"

In [ ]:
with h5py.File(save_file, "w") as file:
    file.create_dataset("BKG_input", data=X_test)
    file.create_dataset("BKG_predicted", data=bkg_prediction)
    for i, sig in enumerate(signal_results):
        file.create_dataset("%s_input" % sig[0], data=sig[1])
        file.create_dataset("%s_predicted" % sig[0], data=sig[2])

## Evaluate results

1. Plot loss distribution after prediction (check loss value for signals)
2. Plot ROC curves - how good is anomaly detection for chosen FPR threshold

# 1.

In [ ]:
def mse_loss(true, prediction):
    loss = tf.reduce_mean(tf.math.square(true - prediction),axis=-1)
    return loss

In [ ]:
# compute loss value (true, predicted)
total_loss = []
total_loss.append(mse_loss(X_test, bkg_prediction.astype(np.float32)).numpy())
for i, signal_X in enumerate(signal_data):
    total_loss.append(
        mse_loss(signal_X, signal_results[i][2].astype(np.float32)).numpy()
    )

In [ ]:
bin_size = 100

plt.figure(figsize=(10, 8))
for i, label in enumerate(signal_labels):
    plt.hist(
        total_loss[i],
        bins=bin_size,
        label=label,
        density=True,
        histtype="step",
        fill=False,
        linewidth=1.5,
    )
plt.yscale("log")
plt.xlabel("Autoencoder Loss")
plt.ylabel("Probability (a.u.)")
plt.title("MSE loss")
plt.legend(loc="best")
plt.show()

# 2.

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
labels = np.concatenate([["Background"], np.array(signal_labels)])

In [ ]:
target_background = np.zeros(total_loss[0].shape[0])

plt.figure(figsize=(10, 8))
for i, label in enumerate(labels):
    if i == 0:
        continue  # background events

    trueVal = np.concatenate(
        (np.ones(total_loss[i].shape[0]), target_background)
    )  # anomaly=1, bkg=0
    predVal_loss = np.concatenate((total_loss[i], total_loss[0]))

    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

    auc_loss = auc(fpr_loss, tpr_loss)

    plt.plot(
        fpr_loss,
        tpr_loss,
        "-",
        label="%s (auc = %.1f%%)" % (label, auc_loss * 100.0),
        linewidth=1.5,
    )

    plt.semilogx()
    plt.semilogy()
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    plt.legend(loc="center right")
    plt.grid(True)
    plt.tight_layout()
plt.plot(np.linspace(0, 1), np.linspace(0, 1), "--", color="0.75")
plt.axvline(
    0.00001, color="red", linestyle="dashed", linewidth=1
)  # threshold value for measuring anomaly detection efficiency
plt.title("ROC AE")
plt.show()